# Simulate Sensor Measurement Data and Publish it to an MQTT Broker.

For testing pruposes of IoT systems and MQTT data transmission it is often convenient to simulate sensor data.

The "sensor" in this case is just a sinosoidal function. Parameters are frequency (in Hz), phase, amplitude and amplidue offset. The variable is time (in secs).  

The sensor has an ID and whenever a sensor measurement is taken the value is transmitted together with sensor_id and timestamp. The timestamp is the wallclock time (system time) of the computer the Python code is executed on.

The message payload is a JSON string (Python dictionary string). It looks like this:

```
{"sensor_id": 22, "ts": "2020-12-04T14:21:56.550990CET", "val": 0.0}
```


## The Timestamp 

The timestamp represents the local time of the measurement. It is transmitted as an ISO time string.

In [1]:
from datetime import datetime

In [2]:
 ts = datetime.now()

In [3]:
ts.timestamp()

1607095379.024291

### Time Zones

#### Central European Time (UTC+1):
https://www.timeanddate.com/time/zones/cet

#### Central European Summer Time (UTC+2):
https://www.timeanddate.com/time/zones/cest


In [4]:
ts.isoformat()+"CET"

'2020-12-04T16:22:59.024291CET'



## MQTT Broker

There are several possibilities to use an MQTT broker:

1. Use eclipse publically accessible broker
1. Install mosquitto MQTT broker locally, i.e. on your computer
1. Use another dedicated MQTT broker, e.g. hsrw.space:1883


## Python MQTT Client Package

In case you have to install it first in Anadonda open an Anaconda prompt and execute

```
conda install -c conda-forge paho-mqtt
```


In [5]:
import paho.mqtt.client as mqtt

### MQTT Broker Connection Parameters

In [6]:

broker = "hsrw.space"
port=1883

username = "user"
password = "mqtt"

myname = "john_doe"
#myname = "mickey_mouse"

# this should be a unique ID.
sensor_id = 22

topic = "/emrp2020/"+myname+"/sensor_sim_db"
print("Topic: ", topic)

Topic:  /emrp2020/john_doe/sensor_sim_db


## The code to return a single sine wave

In [7]:
#import math
def mysin(t=0,A0=1,f=1,k=0,phi=0):
    return A0*math.sin(2*math.pi*f*t)+k

### MQTT Publisher Code

In [8]:
def on_publish(client,userdata,result):             #create function for callback
    print("data published")
    pass

#client1= mqtt.Client(client_id = "Prof. B.'s Publisher")           #create client object
client1= mqtt.Client()           #create client object
client1.on_publish = on_publish                          #assign function to callback

client1.username_pw_set(username = username, password = password)
client1.connect(broker,port) #establish connection

from time import sleep
import math
import json


dt = 1 # sample period (delay) in secs

A1 = 1
f1 = 1/20. # Hz

t = 0 # sec

# while(True):
for i in range(5):
    
    val = mysin(t,A1,f1)
    
    ts = datetime.now().isoformat()+"CET"
    #    ms = int(datetime.now().timestamp()*1000) # milliseconds!
    
    data = {"sensor_id": 22, "ts" : ts, "val" : val}
    
    msg = json.dumps(data)
    print("msg: ", msg)
    ret = client1.publish(topic,msg) 
    print("ret: ", ret)
    
    t += dt
    sleep(dt) # sleep 1 sec
    
ret = client1.disconnect()
print("ret: ", ret)

msg:  {"sensor_id": 22, "ts": "2020-12-04T16:23:06.287420CET", "val": 0.0}
data published
ret:  (0, 1)
msg:  {"sensor_id": 22, "ts": "2020-12-04T16:23:07.287879CET", "val": 0.3090169943749474}
data published
ret:  (0, 2)
msg:  {"sensor_id": 22, "ts": "2020-12-04T16:23:08.290807CET", "val": 0.5877852522924731}
data published
ret:  (0, 3)


KeyboardInterrupt: 